<a href="https://colab.research.google.com/github/Nabarundutta/Amazon_ml/blob/master/NLP3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 23% 6.00M/25.7M [00:00<00:00, 61.2MB/s]
100% 25.7M/25.7M [00:00<00:00, 137MB/s] 


In [ ]:

import zipfile
zip_ref = zipfile.ZipFile('/content/imdb-dataset-of-50k-movie-reviews.zip','r') # Changed zipfile to ZipFile
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
df = pd.read_csv('/content/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df.shape

(50000, 2)

In [ ]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [ ]:
df['review'] = df['review'].str.lower()

In [ ]:
type(df['review'][0])

str

In [ ]:
import re
def remove_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'',text)

In [ ]:
df['review'] = df['review'].apply(remove_tags)

In [ ]:
import re
def remove_url(text):
    pattern = re.compile(r'https?://\S+')
    return pattern.sub(r'',text)

In [ ]:
import string
exclude = string.punctuation

In [ ]:
exclude

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def remove_punc(text):
  return text.translate(str.maketrans('','',exclude))

In [ ]:
df['review'] = df['review'].apply(remove_punc)

In [ ]:
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production the filming tech...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically theres a family where a little boy j...,negative
4,petter matteis love in the time of money is a ...,positive


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def remove_stopwords(text):
    new_text = []
    for word in text.split():
      if word in stopwords.words('english'):
        new_text.append('')
      else:
        new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return " ".join(x)

In [ ]:
df['review'] = df['review'].apply(remove_stopwords)

In [ ]:
# import emoji
# print(emoji.demojize('Python is difficult :disappointed:'))

In [ ]:
df.head()

,review,sentiment
0,one reviewers mentioned watching 1 oz e...,positive
1,wonderful little production filming techniqu...,positive
2,thought wonderful way spend time hot s...,positive
3,basically theres family little boy jake thi...,negative
4,petter matteis love time money visually s...,positive


In [ ]:
df['sentiment'].unique()

array(['positive', 'negative'], dtype=object)

In [42]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [46]:
sen1 = 'hello i just gave you $20 message me on nabarun@gmail.com'
doc1 = nlp(sen1)

In [48]:
def token_text(text):
  doc = nlp(text)
  tokens = [token.text for token in doc]
  return tokens

In [52]:
def batch_apply(df, func, batch_size=1000):
    results = []
    for i in range(0, len(df), batch_size):
        batch = df[i:i+batch_size]
        results.extend(batch.apply(func))
        print(batch)
    return results

In [65]:
df['tokens'] = batch_apply(df['review'], token_text)

0      one    reviewers  mentioned   watching  1 oz e...
1       wonderful little production  filming techniqu...
2       thought    wonderful way  spend time    hot s...
3      basically theres  family   little boy jake thi...
4      petter matteis love   time  money   visually s...
                             ...                        
995    nothing  sacred  ask ernie fosselius  days eve...
996     hated   hate selfaware pretentious inanity  m...
997     usually try   professional  constructive   cr...
998      like   going  see    film history class  som...
999      like  zoology textbook given   depiction  an...
Name: review, Length: 1000, dtype: object
1000     movie  awful  cant even  bothered  write  rev...
1001      movie makers always go   authors work  mean ...
1002     cant believe   praising  movie herein arent t...
1003     film really used  locations well   amazing sh...
1004    strangely enough  movie never made    big scre...
                              ...        

In [68]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
def stem_tokens(tokens):
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return stemmed_tokens


# df['stemmed_tokens'] = df['tokens'].apply(stem_tokens)
df['stemmed_tokens'] = batch_apply(df['tokens'], stem_tokens)

0      [one,    , reviewers,  , mentioned,   , watchi...
1      [ , wonderful, little, production,  , filming,...
2      [ , thought,    , wonderful, way,  , spend, ti...
3      [basically, there, s,  , family,   , little, b...
4      [petter, matteis, love,   , time,  , money,   ...
                             ...                        
995    [nothing,  , sacred,  , ask, ernie, fosselius,...
996    [ , hated,   , hate, selfaware, pretentious, i...
997    [ , usually, try,   , professional,  , constru...
998    [  , like,   , going,  , see,    , film, histo...
999    [  , like,  , zoology, textbook, given,   , de...
Name: tokens, Length: 1000, dtype: object
1000    [ , movie,  , awful,  , ca, nt, even,  , bothe...
1001    [  , movie, makers, always, go,   , authors, w...
1002    [ , ca, nt, believe,   , praising,  , movie, h...
1003    [ , film, really, used,  , locations, well,   ...
1004    [strangely, enough,  , movie, never, made,    ...
                              ...        

In [69]:
df.head()

,review,sentiment,tokens,stemmed_tokens
0,one reviewers mentioned watching 1 oz e...,positive,"[one, , reviewers, , mentioned, , watchi...","[one, , review, , mention, , watch, , 1..."
1,wonderful little production filming techniqu...,positive,"[ , wonderful, little, production, , filming,...","[ , wonder, littl, product, , film, techniqu,..."
2,thought wonderful way spend time hot s...,positive,"[ , thought, , wonderful, way, , spend, ti...","[ , thought, , wonder, way, , spend, time,..."
3,basically theres family little boy jake thi...,negative,"[basically, there, s, , family, , little, b...","[basic, there, s, , famili, , littl, boy, j..."
4,petter matteis love time money visually s...,positive,"[petter, matteis, love, , time, , money, ...","[petter, mattei, love, , time, , money, ,..."


In [63]:
stem_words("i am running")

'i am run'

In [70]:
from sklearn.preprocessing import LabelEncoder

In [71]:
le = LabelEncoder()
le.fit(df['sentiment'])

LabelEncoder()

In [72]:
df['sentiment'] = le.transform(df['sentiment'])

In [73]:
df.head()

,review,sentiment,tokens,stemmed_tokens
0,one reviewers mentioned watching 1 oz e...,1,"[one, , reviewers, , mentioned, , watchi...","[one, , review, , mention, , watch, , 1..."
1,wonderful little production filming techniqu...,1,"[ , wonderful, little, production, , filming,...","[ , wonder, littl, product, , film, techniqu,..."
2,thought wonderful way spend time hot s...,1,"[ , thought, , wonderful, way, , spend, ti...","[ , thought, , wonder, way, , spend, time,..."
3,basically theres family little boy jake thi...,0,"[basically, there, s, , family, , little, b...","[basic, there, s, , famili, , littl, boy, j..."
4,petter matteis love time money visually s...,1,"[petter, matteis, love, , time, , money, ...","[petter, mattei, love, , time, , money, ,..."


In [74]:
df.to_csv('processed_imdb.csv',index=False)